In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pima = pd.read_csv("data/diabetes.csv")

In [ ]:
pima.head()

In [ ]:
pima.shape

In [ ]:
pima['Outcome'].value_counts()

In [ ]:
feature_cols = ['Pregnancies', 'Insulin', 'BMI', 'Age','Glucose','BloodPressure','DiabetesPedigreeFunction']
X = pima[feature_cols] # Features
y = pima.Outcome # Target variable

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X,y)

In [ ]:
y_pred=logreg.predict(X)

In [ ]:
logreg.classes_

In [ ]:
pima.Outcome.value_counts()

In [ ]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y, y_pred)
cnf_matrix
# các cột: là dự đoán; các dòng là thực tế
# thực tế là bệnh: 113+155 = 268
# dự đoán đúng (dương tính): 155 ~ 0.58
# dự đoán sai (âm tính): 113

In [ ]:
TN = 445 # dự đoán đúng trường hợp 0: thực tế không bị bênh và dự đoán cũng không bị bệnh
TP = 155 # dự đoán đúng trường hợp 1: thực tế bị bệnh và dự đoán cũng bị bệnh
FP = 55 # dự đoán sai: thực tế là 0 ==> dự đoán là 1
FN = 113 # dự đoán sai: thực tế là 1 ==> dự đoán là 0

In [ ]:
y_pred_proba = logreg.predict_proba(X)[:,1]
fpr, tpr, thresholds = metrics.roc_curve(y,  y_pred_proba)
auc = metrics.roc_auc_score(y, y_pred_proba)

plt.plot(fpr, tpr, marker='.', label="auc="+str(auc))
plt.plot([0, 1], [0, 1], linestyle='--') # y = x

plt.title("ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc=4)
plt.show()

In [ ]:
thresholds[:10]

In [ ]:
thresholds[-10:]

In [ ]:
thresholds.size

In [ ]:
thresholds.min(), thresholds.max()

In [ ]:
# tham khảo https://www.youtube.com/watch?v=bh36ii3UHGo

In [ ]:
# Sensitivity (độ nhạy): trong số những người mắc bệnh có bao nhiêu người có dự đoán đúng là bệnh
# TPR = TP/(TP+FN)
TPR = TP/(TP+FN) # recall class 1
TPR

In [ ]:
# FPR = FP/(FP+TN)
FPR = FP/(FP+TN)
FPR

In [ ]:
# Specificity (độ đặc hiệu): trong số những người không mắc bệnh có bao nhiêu người có dự đoán sai là bệnh
# FPR = 1 - Specificity
spec = TN/(TN+FP) # recall class 2
FPR = 1 - spec
FPR

In [ ]:
y_pred_proba = logreg.predict_proba(X)[:,1] # cột 1: giá trị được tính khi > threshold --> 1, ngược lại < threshold --> 0
y_pred_proba[:10]

In [ ]:
p_lst = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,1.0] # danh sách các threshold
tpr_lst, fpr_lst = [], []
for p in p_lst:
    y_pred = np.where(y_pred_proba>=p, 1, 0)
    cnf_matrix = metrics.confusion_matrix(y, y_pred)
    TN = cnf_matrix[0,0]
    TP = cnf_matrix[1,1]
    FP = cnf_matrix[0,1]
    FN = cnf_matrix[1,0]
    TPR = TP/(TP+FN)
    FPR = FP/(FP+TN)
    fpr_lst.append(FPR)
    tpr_lst.append(TPR)

In [ ]:
auc = metrics.roc_auc_score(y, y_pred_proba)
plt.plot(fpr_lst, tpr_lst, marker='.', label="auc="+str(auc))
plt.plot([0, 1], [0, 1], linestyle='--')

plt.title("ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

In [ ]:
# AUC > 0.90 ==> model rất tốt
# AUC từ 0.80 - 0.90 ==> model tốt
# AUC từ 0.70 - 0.80 ==> model trung bình
# AUC từ 0.60 - 0.70 ==> model không tốt
# AUC từ 0.50 - 0.60 ==> model không dùng được

## Điều chỉnh ngưỡng

In [ ]:
y_pred_proba = logreg.predict_proba(X)[::,1]
fpr, tpr, thresholds = metrics.roc_curve(y,  y_pred_proba)

In [ ]:
df = pd.DataFrame({'tpr':tpr, 'fpr':fpr, 'p':thresholds})

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# chọn ngưỡng p sao cho tpr>0.8
cond = df['tpr']>0.8
df[cond].sort_values(by='tpr').head(10) # chọn p là 0.30

In [ ]:
# chọn ngưỡng p sao cho fpr<0.4
cond = df['fpr']<0.4
df[cond].sort_values(by='fpr').tail(10) # chọn p là 0.24

In [ ]:
# chọn ngưỡng p sao cho tpr>0.8 và fpr<0.4
cond = (df['tpr']>0.8) & (df['fpr']<0.4)
df[cond]

In [ ]:
m = 0.29
m

In [ ]:
th = m
# dự đoán cho threshold mới
y_pred_proba = logreg.predict_proba(X)[::,1]
y_pred_ = np.where(y_pred_proba>=th, 1, 0)
# tính lại confusion matrix và nhận xét
cnf_matrix_ = metrics.confusion_matrix(y, y_pred_)
cnf_matrix_